In [53]:
import IsingLib
import numpy as np
np.set_printoptions(threshold=1000)
np.set_printoptions(linewidth=200)

# Random Graph

num = 15
value = 31
mat = np.random.randint(-value, value, (num, num))
print(mat)

[[  0  -4 -19   3 -25   8  -2  22 -23 -24  -8  29 -29  18  30]
 [-14 -12 -13   5 -21  -2 -25  16  25  27 -11 -24  -9  -9  13]
 [  4  -1  -1 -24  11 -21  30 -16  -1   8 -30  30  -5 -22 -23]
 [ 11 -19  19  -9 -17  25 -30  13 -14  30 -11  -7  12 -14   2]
 [  1   4 -20   8  -5  30 -22  23 -27 -14 -11 -27 -28   0   3]
 [ -5   2   9  20  -2 -31  13   4 -24  30  25  27 -20   3 -29]
 [ 15 -24  -9 -23  23 -21  27 -20   4  23 -26  -6  -2 -27   6]
 [ -1  26  -3 -10  -6   0  -6  19 -22   3 -20   9 -31  10   8]
 [  2  21 -21 -18  10 -29  -1   9  14 -12  28  22  23  13  12]
 [-24  17  -7  23  14 -29 -28  13   5 -13   5  20  21  11   0]
 [  9  16  -9  25 -30 -29  25  14  10   0 -29  -8   6  13  17]
 [ 25  28 -16  21 -27  13   7 -16  -8   2  14   4 -22 -26 -24]
 [ 27  23 -24  -8  -7   8   7  30 -10 -24 -29  26  14  27  -1]
 [ 23 -30   7 -26   6  26  15 -25 -10  11  -5  29   4  27   4]
 [-18  13 -23  28  23 -17 -12  -3 -18 -16 -21  18 -11 -22  -7]]


In [54]:
vartype = 'BINARY'
# If you make a qubo model, you assign 'BINARY' or 'qubo' to variable vartype.
# if you make a Ising model, you assign 'SPIN' or 'ising' to variable vartype.

graph = IsingLib.Model(vartype)

In [55]:
graph.add_terms(H=mat)
graph.compile()

In [56]:
print(graph.matrix)
s, eng = graph.solve()
print(eng)

[[  0.  -4. -19.   3. -25.   8.  -2.  22. -23. -24.  -8.  29. -29.  18.  30.]
 [  0. -12. -13.   5. -21.  -2. -25.  16.  25.  27. -11. -24.  -9.  -9.  13.]
 [  0.   0.  -1. -24.  11. -21.  30. -16.  -1.   8. -30.  30.  -5. -22. -23.]
 [  0.   0.   0.  -9. -17.  25. -30.  13. -14.  30. -11.  -7.  12. -14.   2.]
 [  0.   0.   0.   0.  -5.  30. -22.  23. -27. -14. -11. -27. -28.   0.   3.]
 [  0.   0.   0.   0.   0. -31.  13.   4. -24.  30.  25.  27. -20.   3. -29.]
 [  0.   0.   0.   0.   0.   0.  27. -20.   4.  23. -26.  -6.  -2. -27.   6.]
 [  0.   0.   0.   0.   0.   0.   0.  19. -22.   3. -20.   9. -31.  10.   8.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  14. -12.  28.  22.  23.  13.  12.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0. -13.   5.  20.  21.  11.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -29.  -8.   6.  13.  17.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4. -22. -26. -24.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  1

In [57]:
s, eng = graph.solve('bf')
print(s, eng)

[(1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0)] -364.0


In [58]:
mat, const = graph.get_params('da')
da = IsingLib.DaFormat(mat, api_key='/Users/daisuke.oku/da_key_2.json', version=2)
job_id = da.da_solver()
da.get_job(job_id)

In [59]:
da.get_energy(job_id)
state_li = da.get_configuration(job_id)
for state in state_li:
    print(graph.energy(state))

-364.0


In [60]:
# TSP
import pyqubo

city = 5
vartype = 'BINARY'
x = pyqubo.Array.create('x', (city, city), vartype)
tsp_model = IsingLib.Model(vartype)
dis_li = [[0, 10, 8, 8, 4], [10, 0, 6, 11, 7], [8, 6, 0, 14, 8], [9, 11, 14, 0, 6], [4, 7, 8, 6, 0]]
distance = np.zeros((city**2, city**2))
for i in range(city):
    point = (i + 1) % city
    distance[i*city: (i+1)*city, point*city: (point+1)*city] = dis_li
time_const = pyqubo.Sum(0, city, lambda i: (pyqubo.Sum(0, city, lambda j: x[i, j]) - 1) ** 2)
city_const = pyqubo.Sum(0, city, lambda j: (pyqubo.Sum(0, city, lambda i: x[i, j]) - 1) ** 2)
tsp_model.add_terms(H_0=distance, H_1=time_const, H_2=city_const)

In [62]:
np.set_printoptions(edgeitems=7)
A = 10
tsp_model.compile(H_1=A, H_2=A)
print(tsp_model.matrix)

[[-20.  20.  20.  20.  20.  20.  10.   8.   8.   4.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.]
 [  0. -20.  20.  20.  20.  10.  20.   6.  11.   7.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.]
 [  0.   0. -20.  20.  20.   8.   6.  20.  14.   8.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.]
 [  0.   0.   0. -20.  20.   9.  11.  14.  20.   6.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.]
 [  0.   0.   0.   0. -20.   4.   7.   8.   6.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.  20.]
 [  0.   0.   0.   0.   0. -20.  20.  20.  20.  20.  20.  10.   8.   8.   4.  20.   0.   0.   0.   0.  20.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. -20.  20.  20.  20.  10.  20.   6.  11.   7.   0.  20.   0.   0.   0.   0.  20.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0. -20.  20.  20.   8.   6.  20.  14.   8.   0.   0.  20.   0.   0.   

In [63]:
state, eng = tsp_model.solve(temperature_start=100000, temperature_end=0.001)
print('')
print(state, )
print('')
ans = state.reshape(city, city)
print(ans)


[0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0]

[[0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]]


In [64]:
mat, const = tsp_model.get_params('da')
da = IsingLib.DaFormat(mat, api_key='/Users/daisuke.oku/da_key_2.json', version=2)
job_id = da.da_solver()
da.get_job(job_id)

In [65]:
da.get_energy(job_id)
state_li = da.get_configuration(job_id)
for state in state_li:
    np_state = np.array(state).reshape(city, city)
    print(np_state, tsp_model.energy(state))

[[0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]] -76.0
[[0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]] -76.0
